# Forecasting School Student Demographic Proportions Over Time

### Intro

This is a notebook demonstrating a simple and flexible forecasting workflow that could be adapted to multiple Machine Learning problems on Arbor BI Connector data model. Using previous 5 years of data I predict the next 3 years of each student demographic proportion.

See the full data model here: https://support.arbor-education.com/hc/en-us/articles/16774873076637-BI-Connector-Version-2-Datasets#01J58GFDKTPFBE7SK14Z0NQFFQ

If you have any questions about this then please ask here: https://arbor-hq.circle.so/c/data-and-bi/

## Data Science Workflow  
==================================================  
**Step: Data Collection**  

Description: Gathering and preparing data  
Key Questions:  
- What data do we have?  
- Is it clean and reliable?  
- Do we need more data?  

--------------------------------------------------  
**Step: Exploratory Data Analysis**

Description: Understanding data patterns and relationships  
Key Questions:  
- What patterns exist in the data?  
- Are there any obvious correlations?  
- What are the data distributions?  

--------------------------------------------------  
**Step: Generate Hypotheses**  

Description: Forming theories about relationships  
Key Questions:  
- What relationships might exist?  
- Which features could be important?  
- What are our assumptions?  

--------------------------------------------------  
**Step: Model/Feature Selection**

Description: Choosing appropriate models and features  
Key Questions:  
- Which models suit our problem?  
- What features should we use?  
- What preprocessing is needed?  

--------------------------------------------------  

**Step: Model Building & Tuning**  
Description: Creating and optimizing models  
Key Questions:  
- How well does the model perform?  
- What parameters work best?  
- Are our hypotheses supported?  


--------------------------------------------------  

**Step: Error Analysis**  
Description: Explore errors for bias or patterns.
- Analyze prediction errors
- Look for patterns in mistakes
- Understand model limitations

--------------------------------------------------  

**Optional:**
Think about scheduling/ productionising.

### Set up Environment

In [ ]:

import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col, year
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression 
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')


# # We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


### Query Training Data From Snowflake

For the training data we'll use yearly proportions for each demographic for the past 5 years. As you can see we are querying this data straight from the BI Connector.



You can replace 'DEMOGRAPHIC__STUDENT__SEN'  with any of the following to predict any of these demographic proportions:



**DEMOGRAPHIC__STUDENT__UK_DFE__DISADVANTAGED
DEMOGRAPHIC__STUDENT__UK_DFE__COMPULSORY_SCHOOL_AGE
DEMOGRAPHIC__STUDENT__OUT_OF_AGE_GROUP_COHORT
DEMOGRAPHIC__STUDENT__SEN
DEMOGRAPHIC__STUDENT__UK_DFE__SERVICE_CHILD
DEMOGRAPHIC__STUDENT__UK_DFE__PUPIL_PREMIUM
DEMOGRAPHIC__STUDENT__GIFTED
DEMOGRAPHIC__STUDENT__IN_YEAR_ADMISSION
DEMOGRAPHIC__STUDENT__UK_DFE__FSM
DEMOGRAPHIC__STUDENT__HAS_MEDICAL_CONDITION
DEMOGRAPHIC__STUDENT__EAL
DEMOGRAPHIC__STUDENT__UK_DFE__MOBILE_Y10_Y11
DEMOGRAPHIC__STUDENT__UK_DFE__EVER_6_SERVICE_CHILD
DEMOGRAPHIC__STUDENT__GIFTED_TALENTED
DEMOGRAPHIC__STUDENT__TALENTED
DEMOGRAPHIC__STUDENT__UK_DFE__TRAVELLER
DEMOGRAPHIC__STUDENT__UK_DFE__EVER_6_FSM
DEMOGRAPHIC__STUDENT__CHILD_PROTECTION
DEMOGRAPHIC__STUDENT__IN_CARE
DEMOGRAPHIC__STUDENT__UK_DFE__MOBILE_Y5_Y6**

In [ ]:
# Query Training Data From Snowflake using 'DEMOGRAPHIC__STUDENT__SEN' 
# Notice we use the student_tags_history and academics_year_enrolments to get historical data.

query = """
    SELECT 
        sa.application_id AS APPLICATION_ID,
        YEAR(sa.start_date) AS YEAR,
        COUNT(DISTINCT sa.student_id) AS TOTAL_STUDENTS,
        COUNT(DISTINCT CASE WHEN st.tag_identifier = 'DEMOGRAPHIC__STUDENT__SEN' 
                            AND (st.end_date IS NULL OR st.end_date >= sa.start_date) 
                            THEN st.student_id END) AS SEN_STUDENTS,
        COUNT(DISTINCT CASE WHEN st.tag_identifier = 'DEMOGRAPHIC__STUDENT__SEN' 
                            AND (st.end_date IS NULL OR st.end_date >= sa.start_date) 
                            THEN st.student_id END) / 
        NULLIF(COUNT(DISTINCT sa.student_id), 0) AS SEN_PROPORTION
    FROM 
        ARBOR_BI_CONNECTOR_PRODUCTION.ARBOR_MIS_ENGLAND_MODELLED.STUDENT_ACADEMIC_YEAR_ENROLMENTS AS sa
    LEFT JOIN 
        ARBOR_BI_CONNECTOR_PRODUCTION.ARBOR_MIS_ENGLAND_MODELLED.STUDENT_TAGS_HISTORY AS st
    ON 
        sa.student_id = st.student_id 
        AND YEAR(sa.start_date) = YEAR(COALESCE(st.start_date, CURRENT_DATE))
    WHERE 
        sa.start_date >= DATEADD(year, -5, CURRENT_DATE())
    GROUP BY 
        sa.application_id, YEAR(sa.start_date)
    ORDER BY 
        sa.application_id, YEAR;
"""

### Data Preprocessing

In [ ]:

# Execute the query and load data into a Snowpark DataFrame
sp_df = session.sql(query)

# Convert Snowpark DataFrame to Pandas DataFrame for further processing
pandas_df = sp_df.toPandas()

# Check if the DataFrame contains data
if pandas_df.empty:
    print("The query returned no data.")

# Sort the DataFrame by 'APPLICATION_ID' and 'YEAR' to maintain a logical order
pandas_df.sort_values(['APPLICATION_ID', 'YEAR'], inplace=True)

In [ ]:
# Let's look at the training data
pandas_df.head()

### Create Synthetic Dataset


An alternative option is to create synthetic data (for example if you want to try out models before using on your own data.) Below is code to create synthetic data in the same structure as BI Connector student demographics.

In [ ]:
# Create synthetic dataset
np.random.seed(42)  # For reproducibility - this means the same data will be created every time. Change random seed to change the synthetic data.

def create_synthetic_data():
    # Create 10 schools (APPLICATION_ID) with 5 years of data each
    application_ids = range(1, 11)
    years = range(2019, 2024)  # 5 years of historical data
    
    data = []
    
    for app_id in application_ids:
        # Generate a base SEN proportion (between 1% and 4%)
        base_proportion = np.random.uniform(0.01, 0.04)
        
        # Add some trend and noise
        trend = np.random.uniform(-0.002, 0.002)  # Small yearly trend
        
        for year in years:
            # Calculate SEN proportion with trend and noise
            noise = np.random.normal(0, 0.001)  # Small random variation
            sen_proportion = base_proportion + trend * (year - 2019) + noise
            
            # Ensure proportion stays within reasonable bounds
            sen_proportion = np.clip(sen_proportion, 0, 1)
            
            data.append({
                'APPLICATION_ID': app_id,
                'YEAR': year,
                'SEN_PROPORTION': sen_proportion
            })
    
    # Convert to DataFrame
    pandas_df = pd.DataFrame(data)
    return pandas_df

# Create synthetic data
synthetic_df = create_synthetic_data()

# Display first few rows and summary statistics
print("First few rows of synthetic data:")
print(synthetic_df.head(10))

print("\nSummary statistics:")
print(synthetic_df.describe())

print("\nUnique schools (APPLICATION_IDs):", len(synthetic_df['APPLICATION_ID'].unique()))
print("Years per school:", len(synthetic_df['YEAR'].unique()))
print("Year range:", synthetic_df['YEAR'].min(), "to", synthetic_df['YEAR'].max())

# Now let's run our models on this synthetic data
# [Previous model code here - do you want me to include the full modeling code as well?]

In [ ]:
# Choose whether you want to use synthetic data or data from the BI Connector.
# If you want to use synthetic data then remove the '#' from the line below: data_to_use = 'synthetic'

data_to_use = 'bi_connector'
# data_to_use = 'synthetic'

if data_to_use == 'bi_connector':
    demographics_df = pandas_df
else:
    demographics_df = synthetic_df

In [ ]:
demographics_df.head()

### Exploratory Data Analysis

Now we'll do some exploratory data analysis to understand the training data we will be using.

In [ ]:
# Create a figure with multiple subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution of SEN Proportions
sns.histplot(data=demographics_df, x='SEN_PROPORTION', bins=20, ax=ax1)
ax1.set_title('Distribution of SEN Proportions', fontsize=12)
ax1.set_xlabel('SEN Proportion')
ax1.set_ylabel('Count')
ax1.grid(True, linestyle='--', alpha=0.7)

# 2. Box plot of SEN Proportions by Year
sns.boxplot(data=demographics_df, x='YEAR', y='SEN_PROPORTION', ax=ax2)
ax2.set_title('SEN Proportions by Year', fontsize=12)
ax2.set_xlabel('Year')
ax2.set_ylabel('SEN Proportion')
ax2.grid(True, linestyle='--', alpha=0.7)

# 3. Line plot showing trends for each school
for app_id in demographics_df['APPLICATION_ID'].unique():
    school_data = demographics_df[demographics_df['APPLICATION_ID'] == app_id]
    ax3.plot(school_data['YEAR'], school_data['SEN_PROPORTION'], 
            marker='o', label=f'School {app_id}')
ax3.set_title('SEN Proportion Trends by School', fontsize=12)
ax3.set_xlabel('Year')
ax3.set_ylabel('SEN Proportion')
ax3.grid(True, linestyle='--', alpha=0.7)
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# 4. Box plot of SEN Proportions by School
sns.boxplot(data=demographics_df, x='APPLICATION_ID', y='SEN_PROPORTION', ax=ax4)
ax4.set_title('SEN Proportions by School', fontsize=12)
ax4.set_xlabel('School ID')
ax4.set_ylabel('SEN Proportion')
ax4.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
# plt.savefig('demographics_data_eda.png', bbox_inches='tight', dpi=300)
# plt.close()

# Print summary statistics
print("\nSummary Statistics:")
print(demographics_df.groupby('YEAR')['SEN_PROPORTION'].describe().round(4))

# Print trend analysis
print("\nTrend Analysis:")
for app_id in demographics_df['APPLICATION_ID'].unique():
    school_data = demographics_df[demographics_df['APPLICATION_ID'] == app_id]
    start_val = school_data.iloc[0]['SEN_PROPORTION']
    end_val = school_data.iloc[-1]['SEN_PROPORTION']
    change = end_val - start_val
    print(f"School {app_id}: Overall change = {change:.4f} ({'+' if change > 0 else ''}{change/start_val*100:.1f}%)")



**Analysing our Data**

Based on our Exploratory data analysis we identify the features (columns) to use to predict our outcome variable and choose which models might be best to use. 


Here is a checklist for what to look for in time series data:

**Trend Analysis:**
Look for a clear, linear trend in the data over time.
Use line plots to visualize the relationship between time and your target variable.

**Seasonality:**
Minimal or no repeating seasonal patterns.

**Stationarity:**
Data doesn't have sudden shifts or trends that vary over time.

**Variance:**
Variance remains relatively constant over time.

Ideal Use Case:
The data has a linear trend and lacks seasonality or irregular fluctuations.
Examples: Steady growth/decline in student demographic proportions.


**Choosing a Model**

Based on understanding our data we can then choose a model. 

For Linear Regression, use it when the data shows a clear linear trend with no seasonality or complex patterns; it's ideal for straightforward, steady trends over time. SARIMA is better for data with clear seasonality and autocorrelation, requiring stationarity (check with ADF and ACF/PACF). Prophet handles non-linear trends and flexible seasonality, robust to outliers and missing data. Exponential Smoothing (Holt-Winters) is best for simpler data with consistent trend and seasonality. Machine Learning models excel at capturing complex, non-linear patterns or multivariate relationships. Choose based on your data's trends, seasonality, and complexity.

For this task we will use linear regression (as most demographic proportions trends are likely to be linear without complex patterns), SARIMA (to demonstrate that this model might not be best suited for this task) and as a benchmark we use a simple average as our third model.

### Define Models

In [ ]:
def train_sarima(app_df):
    """Train SARIMA model and return predictions and MAE"""
    try:
        model = SARIMAX(app_df['SEN_PROPORTION'], 
                       order=(1, 1, 1), 
                       seasonal_order=(0, 0, 0, 0),
                       enforce_stationarity=False,
                       enforce_invertibility=False)
        result = model.fit(disp=0)
        forecast = result.get_forecast(steps=3)
        predictions = forecast.predicted_mean.values  # Convert to numpy array
        mae = mean_absolute_error(app_df['SEN_PROPORTION'], result.fittedvalues)
        
        # Add validation check
        if np.any(predictions < 0) or np.any(predictions > 1):
            return None, None
        return predictions, mae
    except Exception as e:
        print(f"SARIMA error: {e}")
        return None, None

def train_simple_average(app_df):
    """Calculate simple moving average"""
    try:
        mean_value = app_df['SEN_PROPORTION'].mean()
        predictions = np.array([mean_value] * 3)
        mae = mean_absolute_error(app_df['SEN_PROPORTION'], 
                                np.array([mean_value] * len(app_df)))
        return predictions, mae
    except Exception as e:
        print(f"Simple Average error: {e}")
        return None, None

def train_linear_regression(app_df):
    """Simple linear regression projection"""
    try:
        # Need at least 2 points for regression
        if len(app_df) < 2:
            return None, None
            
        X = np.arange(len(app_df)).reshape(-1, 1)
        y = app_df['SEN_PROPORTION'].values
        
        # Check if there's any variation in the data
        if np.all(y == y[0]):
            return np.array([y[0]] * 3), 0
        
        model = LinearRegression()
        model.fit(X, y)
        
        # Project next three years - each year should be different following the trend
        future_X = np.arange(len(app_df), len(app_df) + 3).reshape(-1, 1)
        predictions = model.predict(future_X)
        
        # Print debugging info
        print(f"\nDebug for years {app_df.index[0]} to {app_df.index[-1]}:")
        print(f"Historical values: {y}")
        print(f"Slope (change per year): {model.coef_[0]:.6f}")
        print(f"Intercept: {model.intercept_:.6f}")
        print(f"Predicted next 3 years: {predictions}")
        
        # Ensure predictions stay within reasonable bounds
        min_allowed = max(0, min(y) * 0.5)  # Don't go below 0
        max_allowed = min(1, max(y) * 1.5)  # Don't go above 1 (100%)
        predictions = np.clip(predictions, min_allowed, max_allowed)
        
        # Calculate MAE on training data
        train_predictions = model.predict(X)
        mae = mean_absolute_error(y, train_predictions)
        
        # Validate predictions
        if np.all(predictions <= 0) or np.all(predictions >= 1):
            print("Warning: Predictions out of reasonable range")
            return None, None
            
        return predictions, mae
        
    except Exception as e:
        print(f"Linear Regression error: {e}")
        return None, None

def compare_models(pandas_df):
    """Compare different forecasting models and return predictions"""
    predictions_list = []
    total_apps = len(pandas_df['APPLICATION_ID'].unique())
    
    for idx, app_id in enumerate(pandas_df['APPLICATION_ID'].unique(), 1):
        print(f"\rProcessing application {idx}/{total_apps}", end="")
        
        # Filter for current application
        app_df = pandas_df[pandas_df['APPLICATION_ID'] == app_id].copy()
        app_df.set_index('YEAR', inplace=True)
        
        # Dictionary to store model results
        model_results = {
            'SARIMA': train_sarima(app_df),
            'Simple Average': train_simple_average(app_df),
            'Linear Regression': train_linear_regression(app_df)
        }
        
        # Store predictions for each valid model
        for model_name, (predictions, mae) in model_results.items():
            if predictions is not None and mae is not None:
                # Additional validation check
                if np.all((predictions >= 0) & (predictions <= 1)):
                    forecast_index = [app_df.index[-1] + i for i in range(1, 4)]
                    for year, prediction in zip(forecast_index, predictions):
                        predictions_list.append({
                            'APPLICATION_ID': app_id,
                            'YEAR': int(year),
                            'PREDICTED_SEN_PROPORTION': float(prediction),
                            'MAE': float(mae),
                            'MODEL_USED': model_name
                        })
    
    print("\nProcessing complete!")
    return pd.DataFrame(predictions_list)

### Results

In [ ]:
# Run the comparison
predictions_df = compare_models(demographics_df)

# Display summary of model performance
print("\nModel Usage Summary:")
model_counts = predictions_df['MODEL_USED'].value_counts()
print(model_counts)

print("\nAverage MAE by Model:")
avg_mae = predictions_df.groupby('MODEL_USED')['MAE'].mean()
print(avg_mae)

# Add some basic validation checks
print("\nPrediction Range Check:")
print("Min predicted value:", predictions_df['PREDICTED_SEN_PROPORTION'].min())
print("Max predicted value:", predictions_df['PREDICTED_SEN_PROPORTION'].max())

# # Save results to CSV
# predictions_df.to_csv('sen_predictions_comparison.csv', index=False)
# print("\nResults saved to 'sen_predictions_comparison.csv'")

We are using Mean Absolute Error (MAE) as the metric to measure quality of model.This is quite standard for regression tasks, although Mean Squared Error (MSE) could also be used.

In [ ]:

# Create figure
plt.figure(figsize=(15, 10))

# Create subplot grid
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Boxplot of MAEs by model
sns.boxplot(data=predictions_df, x='MODEL_USED', y='MAE', ax=ax1)
ax1.set_title('Distribution of MAE by Model', fontsize=12, pad=20)
ax1.set_xlabel('Model', fontsize=10)
ax1.set_ylabel('Mean Absolute Error', fontsize=10)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, linestyle='--', alpha=0.7)

# Add individual points
sns.swarmplot(data=predictions_df, x='MODEL_USED', y='MAE', 
             color='red', alpha=0.5, ax=ax1)

# 2. Bar plot with error bars
mae_stats = predictions_df.groupby('MODEL_USED').agg({
    'MAE': ['mean', 'std', 'min', 'max']
})['MAE']

# Plot bars
x = range(len(mae_stats.index))
bars = ax2.bar(x, mae_stats['mean'], 
              yerr=mae_stats['std'],
              capsize=5,
              color=['lightblue', 'lightgreen', 'lightpink'])

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}',
            ha='center', va='bottom')

ax2.set_xticks(x)
ax2.set_xticklabels(mae_stats.index, rotation=45)
ax2.set_title('Average MAE by Model (with Standard Deviation)', fontsize=12, pad=20)
ax2.set_xlabel('Model', fontsize=10)
ax2.set_ylabel('Mean Absolute Error', fontsize=10)
ax2.grid(True, linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()

# # Save plot
# plt.savefig('mae_comparison.png', bbox_inches='tight', dpi=300)
# plt.close()

# Print detailed statistics
print("\nDetailed MAE Statistics:")
print(mae_stats.round(6))

# Add statistical significance test
from scipy import stats

# Perform Kruskal-Wallis H-test
models = predictions_df['MODEL_USED'].unique()
mae_data = [predictions_df[predictions_df['MODEL_USED'] == model]['MAE'] for model in models]
h_stat, p_value = stats.kruskal(*mae_data)

print("\nStatistical Test Results:")
print(f"Kruskal-Wallis H-test:")
print(f"H-statistic: {h_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Pairwise Mann-Whitney U tests if Kruskal-Wallis is significant
if p_value < 0.05:
    print("\nPairwise Model Comparisons (Mann-Whitney U test):")
    for i, model1 in enumerate(models):
        for model2 in models[i+1:]:
            stat, p = stats.mannwhitneyu(
                predictions_df[predictions_df['MODEL_USED'] == model1]['MAE'],
                predictions_df[predictions_df['MODEL_USED'] == model2]['MAE']
            )
            print(f"{model1} vs {model2}:")
            print(f"p-value: {p:.4f}")


### Full Predictions

In [ ]:
predictions_df.head(25)

### Visualise Predictions

In [ ]:

# Create figure with multiple subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 20))

# 1. Box plot of predictions by model
models = predictions_df['MODEL_USED'].unique()
data = [predictions_df[predictions_df['MODEL_USED'] == model]['PREDICTED_SEN_PROPORTION'] 
        for model in models]

box_plot = ax1.boxplot(data, labels=models, patch_artist=True)
# Color each box
colors = ['lightblue', 'lightgreen', 'lightpink']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)

ax1.set_title('Distribution of Predictions by Model', fontsize=12, pad=20)
ax1.set_xlabel('Model', fontsize=10)
ax1.set_ylabel('Predicted SEN Proportion', fontsize=10)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, linestyle='--', alpha=0.7)

# 2. Line plot for each school
# Create a color map for schools
unique_schools = demographics_df['APPLICATION_ID'].unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_schools)))

for i, app_id in enumerate(unique_schools):
    # Historical data
    hist_data = demographics_df[demographics_df['APPLICATION_ID'] == app_id]
    ax2.plot(hist_data['YEAR'], hist_data['SEN_PROPORTION'], 
            marker='o', linestyle='-', color=colors[i], alpha=0.6,
            label=f'School {app_id} - Historical')
    
    # Predictions for each model
    pred_data = predictions_df[predictions_df['APPLICATION_ID'] == app_id]
    for model in pred_data['MODEL_USED'].unique():
        model_pred = pred_data[pred_data['MODEL_USED'] == model]
        ax2.plot(model_pred['YEAR'], model_pred['PREDICTED_SEN_PROPORTION'], 
                marker='s', linestyle='--', color=colors[i], alpha=0.4)

ax2.set_title('Historical Values and Predictions by School', fontsize=12, pad=20)
ax2.set_xlabel('Year', fontsize=10)
ax2.set_ylabel('SEN Proportion', fontsize=10)
ax2.grid(True, linestyle='--', alpha=0.7)

# Add legend with smaller font and to the right
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# 3. MAE comparison
mae_by_model = predictions_df.groupby('MODEL_USED')['MAE'].mean()
x = np.arange(len(mae_by_model))
bars = ax3.bar(x, mae_by_model.values, color=['lightblue', 'lightgreen', 'lightpink'])

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}',
            ha='center', va='bottom')

ax3.set_xticks(x)
ax3.set_xticklabels(mae_by_model.index, rotation=45)
ax3.set_title('Mean Absolute Error by Model', fontsize=12, pad=20)
ax3.set_xlabel('Model', fontsize=10)
ax3.set_ylabel('Mean Absolute Error', fontsize=10)
ax3.grid(True, linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()

# Save main plot
plt.savefig('sen_predictions_visualization.png', bbox_inches='tight', dpi=300)
plt.close()

# Create individual plots for each school
for app_id in unique_schools:
    plt.figure(figsize=(12, 6))
    
    # Historical data
    hist_data = demographics_df[demographics_df['APPLICATION_ID'] == app_id]
    plt.plot(hist_data['YEAR'], hist_data['SEN_PROPORTION'], 
            marker='o', linestyle='-', label='Historical', color='black', linewidth=2)
    
    # Predictions for each model
    pred_data = predictions_df[predictions_df['APPLICATION_ID'] == app_id]
    colors = ['blue', 'green', 'red']  # Different color for each model
    for i, model in enumerate(pred_data['MODEL_USED'].unique()):
        model_pred = pred_data[pred_data['MODEL_USED'] == model]
        plt.plot(model_pred['YEAR'], model_pred['PREDICTED_SEN_PROPORTION'], 
                marker='s', linestyle='--', label=f'{model}',
                color=colors[i], linewidth=2)
    
    plt.title(f'School {app_id}: Historical and Predicted SEN Proportions', fontsize=12, pad=20)
    plt.xlabel('Year', fontsize=10)
    plt.ylabel('SEN Proportion', fontsize=10)
    plt.legend(fontsize=9)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Save individual plot
    # plt.savefig(f'school_{app_id}_predictions.png', bbox_inches='tight', dpi=300)
    # plt.close()



### Error Analysis

Now we can look more in-depth at the kinds of errors our model is making.

In [ ]:
# Create figure with 2 key plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Distribution of predictions by model
sns.boxplot(data=predictions_df, x='MODEL_USED', y='PREDICTED_SEN_PROPORTION', ax=ax1)
ax1.set_title('Distribution of Predictions by Model')
ax1.set_xlabel('Model')
ax1.set_ylabel('Predicted SEN Proportion')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, linestyle='--', alpha=0.7)

# 2. Predictions over time by model
sns.lineplot(data=predictions_df, x='YEAR', y='PREDICTED_SEN_PROPORTION', 
            hue='MODEL_USED', marker='o', ax=ax2)
ax2.set_title('Predictions Over Time by Model')
ax2.set_xlabel('Year')
ax2.set_ylabel('Predicted SEN Proportion')
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

# Print basic statistics
print("\nSummary Statistics by Model:")
stats = predictions_df.groupby('MODEL_USED')['PREDICTED_SEN_PROPORTION'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(4)
print(stats)

# Print predictions range for each year
print("\nPrediction Ranges by Year:")
yearly_stats = predictions_df.groupby(['YEAR', 'MODEL_USED'])['PREDICTED_SEN_PROPORTION'].agg([
    'mean', 'min', 'max'
]).round(4)
print(yearly_stats)

# Compare MAE between models
print("\nMAE Comparison:")
mae_stats = predictions_df.groupby('MODEL_USED')['MAE'].agg([
    'mean', 'std', 'min', 'max'
]).round(4)
print(mae_stats)

We can now analyse this error analysis:

Distribution of Predictions by Model (Left Plot):


SARIMA shows the widest spread, with some outlier predictions as high as 0.8 (80% SEN proportion) which seems unrealistic
Simple Average shows a very tight distribution around a low value
Linear Regression also shows a concentrated distribution near the lower end
Most predictions across all models are clustered in the lower range (likely below 0.1 or 10% SEN)


Predictions Over Time by Model (Right Plot):


SARIMA (blue line) shows concerning behavior:

It has increasing uncertainty over time (widening blue shaded area)
Makes unrealistic predictions going up to 0.5 (50% SEN) by 2027
Shows high volatility in predictions


Simple Average (orange line) remains stable over time

This makes sense as it's using the historical mean
Appears to predict around 2-3% SEN consistently


Linear Regression (green line) stays relatively flat

Predictions appear more conservative and realistic
Similar range to Simple Average



Key Issues and Recommendations:

SARIMA appears unstable and likely inappropriate for this data:

The increasing uncertainty and extreme predictions suggest it's not handling the time series well
Consider removing or recalibrating the SARIMA model


Simple Average and Linear Regression seem more reliable:

Their predictions stay within realistic ranges for SEN proportions
They show more stable behavior over time


Model Selection:

Given this analysis, I would lean towards using Linear Regression
SARIMA should probably be discarded unless it can be significantly improved

### Next Steps and Learning Resources

Here are some resources to learn more about time series forecasting:
- https://www.kaggle.com/learn/time-series
- https://quickstarts.snowflake.com/guide/getting-started-with-snowflake-cortex-ml-forecasting-and-classification/index.html#0

If you have any questions about this notebook please ask them here: https://arbor-hq.circle.so/c/data-and-bi/